<font face="Times New Roman">
<div dir=ltr align=center>
<font color=3C99D size=6>
    HISAT2 NGS Reads Alignment

In [ ]:
from random import randint
from random import choice
import re

def rc(n):
  if n == 'A':
    return 'T'
  elif n == 'C':
    return 'G'
  elif n == 'G':
    return 'C'
  elif n == 'T':
    return 'A'

with open('/content/drive/MyDrive/Bio Algo Ex4/chr5.fa', 'r') as f:
    chr5 = f.read().splitlines()

seq = chr5[201:]

quality = 'DEFHI'

### removing repetitive sequences
regex = "[a-zN]"
for i in range(len(seq)):
  seq[i] = re.sub(regex, "", seq[i])

seq = ''.join(seq)
read_counts = 600000
read_length = 90
paired_distance = 120
fragment_length = 2*read_length + paired_distance
exon_distance = [0, 0, 0, 0, 0, 0, 1e2, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]

fastq1 = []
fastq2 = []

for r in range(int(read_counts/2)):
  s1 = randint(0, len(seq) - (fragment_length + max(exon_distance)) -1)
  e_d = int(choice(exon_distance))
  e_d = randint(e_d/10, e_d)
  s1_l = randint(int(fragment_length / 4), int(fragment_length * 3 / 4))
  s2 = s1 + s1_l + e_d
  s2_l = fragment_length - s1_l
  fragment = seq[s1:s1+s1_l] + seq[s2:s2+s2_l]
  r1 = fragment[:read_length]
  r2 = ''.join([rc(i) for i in fragment[:-read_length-1:-1]])
  r1_s = "@SIM00%sR1: 2:length=%s\n%s\n+\n%s" % (r+1, read_length, r1, ''.join([choice(quality) for _ in range(len(r1))]))
  r2_s = "@SIM00%sR2: 2:length=%s\n%s\n+\n%s" % (r+1, read_length, r2, ''.join([choice(quality) for _ in range(len(r2))]))
  fastq1.append(r1_s)
  fastq2.append(r2_s)

In [ ]:
!wget https://cloud.biohpc.swmed.edu/index.php/s/fE9QCsX3NH4QwBi/download
!unzip download

with open('/content/hisat2-2.2.1/chr5_R1.fq', 'w') as f:
    f.write('\n'.join(fastq1))

with open('/content/hisat2-2.2.1/chr5_R2.fq', 'w') as f:
    f.write('\n'.join(fastq2))

%cd hisat2-2.2.1/
!make
!export PATH="/content/hisat2-2.2.1/:$PATH"
!./hisat2-build "/content/drive/MyDrive/Bio Algo Ex4/chr5.fa" "ht2_index_output"
!./hisat2 -x ht2_index_output -1 chr5_R1.fq -2 chr5_R2.fq -S "ex4_sam_output.sam"

300000 reads; of these:
  300000 (100.00%) were paired; of these:
    61531 (20.51%) aligned concordantly 0 times
    235519 (78.51%) aligned concordantly exactly 1 time
    2950 (0.98%) aligned concordantly >1 times
    ----
    61531 pairs aligned concordantly 0 times; of these:
      36840 (59.87%) aligned discordantly 1 time
    ----
    24691 pairs aligned 0 times concordantly or discordantly; of these:
      49382 mates make up the pairs; of these:
        23845 (48.29%) aligned 0 times
        24193 (48.99%) aligned exactly 1 time
        1344 (2.72%) aligned >1 times
96.03% overall alignment rate
